In [1]:
import pandas as pd
from database_utils import DatabaseConnector
from data_extraction import DataExtractor
from data_cleaning import DataCleaning

In [4]:
rds_connector = DatabaseConnector('db_creds.yaml')
local_connector = DatabaseConnector('sales_data.yaml')


In [5]:
df = DataExtractor.read_rds_tables(rds_connector.engine)

In [4]:
print(df)

       index first_name last_name date_of_birth                       company  \
0          0   Sigfried     Noack    1990-09-30            Heydrich Junitz KG   
1          1        Guy     Allen    1940-12-01                       Fox Ltd   
2          2      Harry  Lawrence    1995-08-02     Johnson, Jones and Harris   
3          3     Darren   Hussain    1972-09-23                   Wheeler LLC   
4          4      Garry     Stone    1952-12-20                    Warner Inc   
...      ...        ...       ...           ...                           ...   
15315  14913    Stephen   Jenkins    1943-08-09  Thornton, Carroll and Newman   
15316  14994    Stephen     Smith    1948-08-20               Robinson-Harris   
15317  15012    Stephen  Losekann    1940-10-09                       Rosenow   
15318  15269    Stephen    Rivera    1952-06-04         Taylor, Fry and Jones   
15319   1249    Stephen    Duncan    1994-03-27    Phillips, Brown and Powell   

                      email

### Data Cleaning - User Data
- Understand the shape and types of data (change if necessary).
- Begin cleaning the user data, look out for NULL values, errors with dates, incorrectly typed values and rows filled with the wrong information.


In [5]:
print(df.shape)
print(df.info())

(15320, 12)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          15320 non-null  int64 
 1   first_name     15320 non-null  object
 2   last_name      15320 non-null  object
 3   date_of_birth  15320 non-null  object
 4   company        15320 non-null  object
 5   email_address  15320 non-null  object
 6   address        15320 non-null  object
 7   country        15320 non-null  object
 8   country_code   15320 non-null  object
 9   phone_number   15320 non-null  object
 10  join_date      15320 non-null  object
 11  user_uuid      15320 non-null  object
dtypes: int64(1), object(11)
memory usage: 1.4+ MB
None


In [6]:
print(df['country_code'].value_counts())

country_code
GB            9365
DE            4708
US            1205
NULL            21
GGB              6
5D74J6FPFJ       1
XPVCZE2L8B       1
QREF9WLI2A       1
XKI9UXSCZ1       1
RVRFD92E48       1
IM8MN1L9MJ       1
LZGTB0T5Z7       1
FB13AKRI21       1
OS2P9CMHR6       1
NTCGYW8LVC       1
PG8MOC0UZI       1
0CU6LW3NKB       1
QVUW9JSKY3       1
VSM4IZ4EL3       1
44YAIDY048       1
Name: count, dtype: int64


In [7]:
clean_df = DataCleaning()

clean_df.drop_nulls(df)

In [7]:
df['country_code'] = df['country_code'].replace(['GGB'], 'GB')

In [8]:
search = ['GB','US', 'DE']
df = df[df['country_code'].isin(search)]
print(df['country_code'].value_counts())

country_code
GB    9371
DE    4708
US    1205
Name: count, dtype: int64


In [9]:
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], errors='coerce')  

df['join_date'] = pd.to_datetime(df['join_date'], errors='coerce')  

print(df.shape)
print(df.info())

(15284, 12)
<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          15284 non-null  int64         
 1   first_name     15284 non-null  object        
 2   last_name      15284 non-null  object        
 3   date_of_birth  15257 non-null  datetime64[ns]
 4   company        15284 non-null  object        
 5   email_address  15284 non-null  object        
 6   address        15284 non-null  object        
 7   country        15284 non-null  object        
 8   country_code   15284 non-null  object        
 9   phone_number   15284 non-null  object        
 10  join_date      15261 non-null  datetime64[ns]
 11  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), int64(1), object(9)
memory usage: 1.5+ MB
None


In [10]:
df['phone_number'] = df['phone_number'].str.replace(r'\D+', '', regex=True).astype('float')
df['phone_number'] = pd.to_numeric(df['phone_number'], errors='coerce')  
df.dropna(subset=['phone_number'], inplace=True)

print(df.shape)
print(df.info())

(15284, 12)
<class 'pandas.core.frame.DataFrame'>
Index: 15284 entries, 0 to 15319
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   index          15284 non-null  int64         
 1   first_name     15284 non-null  object        
 2   last_name      15284 non-null  object        
 3   date_of_birth  15257 non-null  datetime64[ns]
 4   company        15284 non-null  object        
 5   email_address  15284 non-null  object        
 6   address        15284 non-null  object        
 7   country        15284 non-null  object        
 8   country_code   15284 non-null  object        
 9   phone_number   15284 non-null  float64       
 10  join_date      15261 non-null  datetime64[ns]
 11  user_uuid      15284 non-null  object        
dtypes: datetime64[ns](2), float64(1), int64(1), object(8)
memory usage: 1.5+ MB
None


In [11]:
local_connector.upload_to_db(df, 'dim_users')